## Run "MuMo-CoCo" Excel CO<sub>2</sub> transportation cost model

https://docs.xlwings.org/en/stable/quickstart.html

### import modules

In [ ]:
import numpy as np
import pandas as pd
import xlwings as xw

### constants

In [ ]:
# molar mass (kg/kmol)
molar_mass_H2_kg_per_kmol = 2.016
molar_mass_CO2_kg_per_kmol = 44.009

# stoichiometric ratio (mol/mol)
stoic_mol_H2_per_mol_CO2 = 1.0

# kg per tonne
kg_per_tonne = 1000.0

# tonnes per kt
tonne_per_kt = 1000.0

# days per year
# implicit assumption: 100% capacity factor
day_per_yr = 365.0

### user input

In [ ]:
# average inflation rate 
# used in the CO2 transportation model
avg_infl = 0.023

# output dollar year
output_dollar_year = 2022

In [ ]:
# hydrogen refueling station capacity (kg H2/day)
stn_cap_kg_H2_per_day = np.concatenate((
    np.arange(50, 1000, 50),
    np.arange(1000, 2501, 100)
))

stn_cap_kg_H2_per_day

In [ ]:
# transport distance (mile)
distance_mi = np.concatenate((
    np.arange(10, 100, 10),
    np.arange(100, 1001, 100)
))

distance_mi

### calculations

In [ ]:
# calculate CO2 flowrate to recycle (ktCO2/yr)
# assuming perfect reactions, no losses, etc.
size_kt_co2_per_yr = stn_cap_kg_H2_per_day * day_per_yr * \
    molar_mass_CO2_kg_per_kmol / molar_mass_H2_kg_per_kmol / \
    stoic_mol_H2_per_mol_CO2 / kg_per_tonne / tonne_per_kt

size_kt_co2_per_yr

### run model and write output

In [ ]:
# create lists
list_input_values = []
list_gross_costs = []

In [ ]:
# open workbook 
wb = xw.Book('MuMo-CoCo_v0_S2A.xlsm')

# instantiate "user interface" sheet
ui = wb.sheets['User Interface']

In [ ]:
# update number format
ui['F6:F9'].number_format = 'general'

In [ ]:
# calculate conversion (multiplier) from input dollar year to 
# output dollar year
dollar_year_multiplier = (
    1 + avg_infl)**(output_dollar_year - ui['start_year'].value)

In [ ]:
# read in input and result names
input_names = ui['A4:A30'].value
gross_cost_names = ui['D6:D9'].value

# update result names
gross_cost_names = [
    name + ' ($/t-CO2 gross)' for name in gross_cost_names]

In [ ]:
input_names

In [ ]:
gross_cost_names

In [ ]:
# run model for user-defined flowrates and distances
for ui['size_kt_co2_per_yr'].value in size_kt_co2_per_yr:
    for ui['distance_mi'].value in distance_mi:
        
        # print message
        print('Running: {:0.1f} ktCO2/yr, {:0.0f} miles...'.format(
            ui['size_kt_co2_per_yr'].value, 
            ui['distance_mi'].value))
        
        # convert cost results to output dollar year
        gross_costs = \
            ui['F6:F9'].options(np.array).value * dollar_year_multiplier
        
        # append results
        list_input_values.append(ui['B4:B30'].value)
        list_gross_costs.append(gross_costs)
        
print('Done.')

In [ ]:
# create dataframe for inputs
df_inputs = pd.DataFrame(
    list_input_values, 
    columns=input_names
)

# create dataframe for results
df_results = pd.DataFrame(
    list_gross_costs, 
    columns=gross_cost_names
)

# concatenate dataframes
df = pd.concat([df_inputs, df_results], axis='columns')

# add output dollar year column
df['Output Dollar Year (User Input)'] = output_dollar_year
df = df[[df.columns[-1]] + df.columns[:-1].tolist()]

# save dataframe
df.to_csv('../liq_co2_trucking_costs.csv', index=False)

In [ ]:
df.columns.tolist()